# 第五题：实现带有拉普拉斯修正的朴素贝叶斯

实验内容：
1. 叙述拉普拉斯修正的作用
2. 给出使用的数据集
3. 给出实现的代码，要有详细的注释
4. 给出模型评价指标的结果

# 拉普拉斯修正的作用

训练集上，很多样本的取值可能并不在其中，但是这不并代表这种情况发生的概率为0，因为未被观测到，并不代表出现的概率为0.  
如果属性取值组合在训练集中并未出现过，朴素贝叶斯分类后可能某个属性取值的信息抹掉了其他属性的取值，在概率估计时，通常解决这个问题的方法是要进行平滑处理，常用拉普拉斯修正.  
拉普拉斯修正的含义是，在训练集中总共的分类数，用 N 表示；di 属性可能的取值数用 Ni 表示，因此原来的先验概率 P(c) 的计算公式由：  
P(c) = Dc / D  
被拉普拉斯修正为  
P(c) = (Dc + 1) / (D + N)  
类的条件概率由P(xi|c) = Dc,xi / Dc  
被拉普拉斯修正为  
P(xi|c) = (Dc,xi + 1) / (Dc + Ni)  

# 1. 导入数据集

In [1]:
import numpy as np
spambase = np.loadtxt('data/spambase/spambase.data', delimiter = ",")
spamx = spambase[:, :57]
spamy = spambase[:, 57]

## 二分类将数据集变成离散的

In [2]:
spamx_binary = (spamx != 0).astype('int')

# 2. 划分数据集

In [3]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(spamx_binary, spamy, test_size = 0.4, random_state = 32)

In [4]:
trainX.shape, trainY.shape, testX.shape, testY.shape

((2760, 57), (2760,), (1841, 57), (1841,))

In [5]:
print(testX)

[[0 0 0 ... 1 1 1]
 [0 0 1 ... 1 1 1]
 [1 0 1 ... 1 1 1]
 ...
 [0 0 1 ... 1 1 1]
 [0 0 0 ... 1 1 1]
 [0 0 0 ... 1 1 1]]


# 3.实现带拉普拉斯修正的朴素贝叶斯

In [6]:
import math

In [18]:
class myGaussianNB:
    '''
    处理离散特征的朴素贝叶斯
    '''
    def __init__(self):
        '''
        初始化四个字典
        self.label_mapping     类标记 与 下标(int)
        self.probability_of_y  类标记 与 先验概率(float)
        self.probability_of_attributes 类标记 与 属性列表的概率(float)
        '''
        self.label_mapping = dict()
        self.probability_of_y = dict()
        self.probability_of_attributes = dict()
        
    def _clear(self):
        '''
        为了防止一个实例反复的调用fit方法，我们需要每次调用fit前，将之前学习到的参数删除掉
        '''
        self.label_mapping.clear()
        self.probability_of_y.clear()
        self.probability_of_attributes.clear()
    
    def fit(self, trainX, trainY):
        '''
        这里，我们要根据trainY内的类标记，针对每类，计算这类的先验概率
        Parameters
        ----------
            trainX: np.ndarray, 训练样本的特征, 维度：(样本数, 特征数)
        
            trainY: np.ndarray, 训练样本的标记, 维度：(样本数, )
        '''
        
        # 先调用_clear
        self._clear()
        
        # 获取类标记
        labels = np.unique(trainY)
        
        # 添加类标记与下标的映射关系
        self.label_mapping = {label: index for index, label in enumerate(labels)}
        
        # 遍历每个类
        for label in labels:
            # 取出为label这类的所有训练样本，存为 x
            x = trainX[trainY == label, :]
            # 计算先验概率，用 x 的样本个数除以训练样本总个数，存储到 self.probability_of_y 中，键为 label，值为先验概率,利用拉普拉斯变换
            self.probability_of_y[label] = (x.shape[0] + 1) / (trainX.shape[0] + labels.shape[0])
            #result用来存放分类为label的第index个属性值为0,1的概率
            result = dict()
            tmp = []
            for index in range(trainX.shape[1]):
                y = [v for v in x[:,index] if v == 0]
                #分类为label的第index个属性,属性值为0的样本占分类为label的所有样本的概率
                pro = (len(y) + 1) / (x.shape[0] + 2)
                tmp.append(pro)
                #分类为label的第index个属性,属性值为1的样本占分类为label的所有样本的概率
                y = [v for v in x[:,index] if v == 1]
                pro = (len(y) + 1) / (x.shape[0] + 2) 
                tmp.append(pro)
                result[index] = tmp
                
            self.probability_of_attributes[label] = result
          
        
    def predict(self, testX):
        '''
        给定测试样本，预测测试样本的类标记，这里我们要实现化简后的公式

        Parameters
        ----------
            testX: np.ndarray, 测试的特征, 维度：(测试样本数, 特征数)
    
        Returns
        ----------
            prediction: np.ndarray, 预测结果, 维度：(测试样本数, )
        '''
        
        # 初始化一个空矩阵 results，存储每个样本属于每个类的概率，维度是 (测试样本数，类别数)，每行表示一个样本，每列表示一个特征
        results = np.empty((testX.shape[0], len(self.probability_of_y)))

        # 初始化一个列表 labels，按 self.label_mapping 的映射关系存储所有的标记，一会儿会在下面的循环内部完成存储
        labels = [0] * len(self.probability_of_y)
        
        # 遍历当前的类，label为类标记，index为下标，我们将每个样本预测出来的这个 label 的概率，存到 results 中的第 index 列
        for label, index in self.label_mapping.items():
            
            # 先验概率存为 py
            py = self.probability_of_y[label]
            
            # 计算所有特征的条件概率之积,利用取对数防止下溢
            result = []
            for i in range(testX.shape[0]):
                tmp = []
                for j in range(testX.shape[1]):
                    #label类第j个属性,值为testX[i][j]的概率
                    tmp.append(self.probability_of_attributes[label][j][testX[i][j]])
                result.append(np.sum(np.log(tmp)) + np.log(py))
            
            # 将所有测试样本属于当前这类的概率，存入到results中
            results[:, index] = result
            # 将当前的label，按index顺序放入到labels中
            labels[index] = label
    
        # 将labels转换为np.ndarray
        np_labels = np.array(labels)
        
        # 循环结束后，就计算出了给定测试样本，当前样本属于这类的概率的近似值，存放在了results中，每行对应一个样本，每列对应一个特征
        # 我们要求每行的最大值对应的下标，也就是求每个样本，概率值最大的那个下标是什么，结果存入max_prob_index中
        
        max_prob_index = np.argmax(results,axis = 1)
        # debug
        assert max_prob_index.shape == (len(testX), )
        
        # 现在得到了每个样本最大概率对应的下标，我们需要把这个下标变成 np_labels 中的标记
        # 使用上面小技巧中的第五点求解
       
        prediction = np_labels[max_prob_index]
        
        # debug
        assert prediction.shape == (len(testX), )
        
        # 返回预测结果
        return prediction

In [22]:
# 测试样例
from sklearn.metrics import accuracy_score
model = myGaussianNB()
model.fit(trainX, trainY)
accuracy_score(testY, model.predict(testX)) 

0.6686583378598587

# 4. 计算其他的指标

###### 双击此处填写

查准率|查全率|F1
-|-|-
0.5787|0.5267|0.5515

In [25]:
# YOUR CODE HERE
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
precision = precision_score(testY,model.predict(testX))
recall = recall_score(testY,model.predict(testX))
f1 = f1_score(testY,model.predict(testX))

In [26]:
print("朴素贝叶斯分类器在测试集上的三项指标")
print("查准率:",precision)
print("查全率:",recall)
print("f1值:",f1)
print()

朴素贝叶斯分类器在测试集上的三项指标
查准率: 0.5787037037037037
查全率: 0.526685393258427
f1值: 0.5514705882352942

